## Introduction
------------
### 1. Environment & Framework
The following code is implemented with PyTorch 1.7.1 under Ubuntu 18.04.5. <br>

### <br> 2. Pipeline at a Glance
The entire code is broken down into three parts.

1. Dataset <br>
```Dataset``` is to convert the raw ```.csv``` file into ```torch.Tensor```, and it also divide the original set into train and test sets. Default ratio for test set is set to 0.3. <br>
<br>
2. Builder <br>
Without model itself, ```Builder``` is the core role in this implementation. ```Builder``` not only executes learning the model, but has some additional features to plot our prediction against ground truth. Considering that this work is a regression problem, visualization can be sometimes useful to acknowledge if the model is seemingly accurate. Though, ```Builder``` also outputs renouned metrics such as MAE.<br>
<br>
3. Model <br>
Defines the models to implement.

In [ ]:
import os
from datetime import datetime, date
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as fn
import torch.optim as optim
from torchvision import transforms

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
plt.style.use('ggplot')

### 1. Dataset

In [ ]:
class Dataset:
    def __init__(self, raw, test_size=0.3, random_state=42):
        self.scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
        self.raw = raw.values #shape: (-1, f+t), ndarray
        self.tr, self.tst = train_test_split(self.raw.copy(), test_size=test_size, random_state=random_state)
        self.tr_M, self.tr_m = torch.from_numpy(self.tr.max(axis=1)).float(), torch.from_numpy(self.tr.min(axis=1)).float()
        self.tst_M, self.tst_m = torch.from_numpy(self.tst.max(axis=1)).float(), torch.from_numpy(self.tst.min(axis=1)).float()
        
        for i in range(self.tr.shape[0]):
            self.tr[i] = self.scaler.fit_transform(self.tr[i].reshape(-1,1)).reshape(1,-1)
        for i in range(self.tst.shape[0]):
            self.tst[i] = self.scaler.fit_transform(self.tst[i].reshape(-1,1)).reshape(1,-1)
            
        self.tr, self.tst = torch.from_numpy(self.tr).float(), torch.from_numpy(self.tst).float()

### 2. Builder

In [ ]:
class Builder:
    def __init__(self, model, dataset, f, t):
        self.model = model
        self.dataset = dataset
        self.f, self.t = f, t
        self.tr_loss, self.val_loss = [], []
    
    
    def _mini_batch(self, x, B, shuffle=True): #shape: (-1, f+t), tensor
        batch = []
        if (x.size(0) % B) == 0:
            iteration = x.size(0) // B
        else:
            iteration = x.size(0) // B + 1
        
        order = np.arange(x.size(0))
        if shuffle:
            np.random.shuffle(order)
            
        for i in range(iteration):
            indices = order[B*i : B*(i+1)]
            batch.append(x[indices])
        return batch #shape: (iteration), element_shape: (B, f+t)
    
    
    ####################TRAIN####################
    def fit(self, optimizer=None, loss_fn=None, batch_size=128, lr=1e-3, epochs=10, val_ratio=0.1):
        # mini batch -> val -> train -> print
        data_load = self._mini_batch(self.dataset.tr, batch_size, shuffle=True)
        tr_load = data_load[int(len(data_load)*val_ratio):]
        val_load = data_load[:int(len(data_load)*val_ratio)]
        
        loss_fn = loss_fn()
        opt = optimizer(self.model.parameters(), lr=lr)
        
        for ep in range(epochs): #x_shape: (B, f), y_shape: (B, t)
            tr_loss_bin, val_loss_bin = [], []
            
            with torch.no_grad(): #cross validate
                for i, val_data in enumerate(tqdm(val_load)): 
                    x, y = val_data[:, :self.f], val_data[:, self.f:]
                    self.model.eval()
                    pred = self.model(x.to(device)) # GPU
                    loss = loss_fn(pred, y.to(device))
                    val_loss_bin.append(loss.item())
                    
            for i, tr_data in enumerate(tqdm(tr_load)): # train
                x, y = tr_data[:, :self.f], tr_data[:, self.f:]
                self.model.train()
                pred = self.model(x.to(device)) # GPU
                loss = loss_fn(pred, y.to(device))
                tr_loss_bin.append(loss.item())
                loss.backward()
                opt.step()
            
            self.tr_loss.append(np.mean(tr_loss_bin))
            self.val_loss.append(np.mean(val_loss_bin))
            print({scheduler.get_lr()})
            print(f'Epoch: {ep+1}/{epochs}, Train Loss: {self.tr_loss[-1]:.4f}, Val Loss: {self.val_loss[-1]:.4f}')
    
    
    ####################EVALUATION####################
    def eval(self):
        MAE = nn.L1Loss()
        MSE = nn.MSELoss()
        MAE_bin, MSE_bin = [], []
        data_load = self._mini_batch(self.dataset.tst, 128, shuffle=False)
        M_load = self._mini_batch(self.dataset.tst_M, 128, shuffle=False)
        m_load = self._mini_batch(self.dataset.tst_m, 128, shuffle=False)
        
        for i, data in enumerate(tqdm(data_load)):
            x, y = data[:, :self.f], data[:, self.f:] #(iteration), (B, f+t)
            M, m = M_load[i].view(-1,1), m_load[i].view(-1,1) #(iteration), (B, 1)
            self.model.eval()
            pred = self.model(x.to(device)).to('cpu')            
            pred, y = (M-m)*pred + m, (M-m)*y + m
            MAE_bin.append(MAE(pred, y).item())
            MSE_bin.append(MSE(pred, y).item())
        MAE_result = np.mean(MAE_bin)
        RMSE_result = np.sqrt(np.mean(MSE_bin))
        print(f'MAE Loss: {MAE_result:.2f}, RMSE Loss: {RMSE_result:.2f}')

    
    ####################VISUALIZATION####################
    def plot(self, row, col, train=False):
        if train:
            order = np.arange(self.dataset.tr.size(0))
            np.random.shuffle(order)
            order = order[:row*col]
            #shape: (row*col, f+t)
            data, M, m = self.dataset.tr[order], self.dataset.tr_M[order].view(-1,1), self.dataset.tr_m[order].view(-1,1)
        else:
            order = np.arange(self.dataset.tst.size(0))
            np.random.shuffle(order)
            order = order[:row*col]
            #shape: (row*col, f+t)
            data, M, m = self.dataset.tst[order], self.dataset.tst_M[order].view(-1,1), self.dataset.tst_m[order].view(-1,1)
            
        with torch.no_grad():
            self.model.eval()
            x, y = data[:, :self.f], data[:, self.f:] #x_shape: (row*col, f), y_shape: (row*col, t)
            pred = self.model(x.to(device)).to('cpu') #pred_shape: (row*col, t)
            pred, y = (M-m)*pred + m, (M-m)*y + m
            
        fig = plt.figure(figsize=(20,15))
        for i in range(row*col):
            ax = plt.subplot(row, col, i+1)
            ax.plot(np.arange(self.t)+1, pred[i], label='Prediction')
            ax.plot(np.arange(self.t)+1, y[i], label='Truth')
            ax.legend(loc='upper left')
        
        
    def learning_curve(self):
        fig = plt.figure(figsize=(10,5))
        plt.title('Learning Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.plot(np.arange(len(self.tr_loss))+1, self.tr_loss, label='Train')
        plt.plot(np.arange(len(self.val_loss))+1, self.val_loss, label='Val')
        plt.xlim((1, len(self.tr_loss)))
        plt.legend()
    
    
    ####################SAVE & LOAD MODEL####################
    def save_model(self, name):
        torch.save(self.model.state_dict(), os.path.join('./models', name+'.pt'))
        np.save(os.path.join('./history', name+' (Train)'), self.tr_loss)
        np.save(os.path.join('./history', name+' (Val)'), self.val_loss)
        
    
    def load_model(self, name):
        self.model.load_state_dict(torch.load(os.path.join('./models', name+'.pt')))
        self.tr_loss = list(np.load(os.path.join('./history', name+' (Train).npy')))
        self.val_loss = list(np.load(os.path.join('./history', name+' (Val).npy')))

### 3. Model

In [ ]:
class fourier(nn.Module):
    def __init__(self, depth, t):
        super().__init__()
        self.A = nn.parameter.Parameter(torch.rand(depth, t))
        self.B = nn.parameter.Parameter(torch.rand(depth, t))
        self.C = nn.parameter.Parameter(torch.rand(depth, t))
        
    def forward(self, x): # in:(B, detph, t), out:(B, t)
        x = torch.mul(x, self.B)
        x = torch.add(x, self.C)
        x = torch.sin(x)
        x = torch.mul(x, self.A)
        x = torch.sum(x, 1)
        return x

In [ ]:
class Model(nn.Module):
    def __init__(self, filter_size, target_size, depth=32):
        super().__init__()
        self.f = filter_size
        self.t = target_size
        self.k = filter_size - target_size + 1
        self.d = depth
        self.conv = nn.Conv1d(1, depth, self.k, 1) #in: (B, 1, f), #out: (B, depth, t)
        self.fourier = fourier(depth, t) # in:(B, detph, t), out:(B, t)
        
    def forward(self, x): #in: (B, f)
        x = x.unsqueeze(1) # (B, 1, f)
        x = self.conv(x) # (B, depth, t)
        x = self.fourier(x) # (B, t)
        return x

### 4. Execution & Visualization

#### Data Preperation

In [ ]:
f, t = 90, 30
raw = pd.read_csv(f'./data/Data_({f},{t})').iloc[:,1:]#.values

In [ ]:
dataset = Dataset(raw)

#### Model and Builder

In [ ]:
model = Model(f, t, depth=32).to(device)

In [ ]:
builder = Builder(model, dataset, f, t)

#### Training

In [ ]:
builder.fit(optimizer=optim.Adam, loss_fn=nn.L1Loss, batch_size=128, lr=1e-5, epochs=50, val_ratio=0.1)

#### Evaluation

In [ ]:
builder.learning_curve()

In [ ]:
builder.plot(6,2, train=False)

In [ ]:
builder.eval()

##### Save Model

In [ ]:
#builder.save_model(f'CNN+T2V - ({f},{t})')
#builder.load_model(f'CNN+T2V - ({f},{t})')